In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import warnings
warnings.filterwarnings('ignore')

In [4]:
from transformers import BertTokenizer, LineByLineTextDataset

vocab_file_dir = '/content/bangla-vocab.txt' 

tokenizer = BertTokenizer.from_pretrained(vocab_file_dir)

sentence = 'শেষ দিকে সেনাবাহিনীর সদস্যরা এসব ঘর তাঁর প্রশাসনের কাছে হস্তান্তর করেন'

encoded_input = tokenizer.tokenize(sentence)
print(encoded_input)

['শেষ', 'দিকে', 'সেনাবাহিনীর', 'সদসযরা', 'এসব', 'ঘর', 'তার', 'পরশাসনের', 'কাছে', 'হসতানতর', 'করেন']


In [5]:
dataset= LineByLineTextDataset(tokenizer = tokenizer,file_path = '/content/raw_bangla_for_BERT.txt',block_size = 128 )

print('No. of lines: ', len(dataset))

No. of lines:  86518


In [6]:
from transformers import BertConfig, BertForMaskedLM, DataCollatorForLanguageModeling

config = BertConfig(vocab_size=50000,hidden_size=768,num_hidden_layers=6,num_attention_heads=12,max_position_embeddings=512)
 
model = BertForMaskedLM(config)
print('No of parameters: ', model.num_parameters())


data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

No of parameters:  81965648


In [7]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(output_dir='/content/bert_output',overwrite_output_dir=True,num_train_epochs=1,per_device_train_batch_size=32,
save_steps=10_000,save_total_limit=2)

In [10]:
trainer = Trainer(model=model,args=training_args,data_collator=data_collator,train_dataset=dataset)

In [11]:
trainer.train()
trainer.save_model('/content/model_save')

Step,Training Loss
500,4.771900
1000,3.077000
1500,2.981500
2000,2.839200
2500,2.902900


In [12]:
from transformers import pipeline

model = BertForMaskedLM.from_pretrained('/content/model_save')

fill_mask = pipeline("fill-mask",model=model,tokenizer=tokenizer)



In [13]:
fill_mask('লাশ উদ্ধার করে ময়নাতদন্তের জন্য কক্সবাজার [MASK] মর্গে পাঠিয়েছে পুলিশ')

[{'score': 0.03529264032840729,
  'sequence': 'লাশ উদধার করে মযনাতদনতের জনয ককসবাজার বলে মরগে পাঠিযেছে পলিশ',
  'token': 614,
  'token_str': 'ব ল ে'},
 {'score': 0.030674153938889503,
  'sequence': 'লাশ উদধার করে মযনাতদনতের জনয ককসবাজার করা মরগে পাঠিযেছে পলিশ',
  'token': 604,
  'token_str': 'ক র া'},
 {'score': 0.01854892447590828,
  'sequence': 'লাশ উদধার করে মযনাতদনতের জনয ককসবাজার করে মরগে পাঠিযেছে পলিশ',
  'token': 542,
  'token_str': 'ক র ে'},
 {'score': 0.013584140688180923,
  'sequence': 'লাশ উদধার করে মযনাতদনতের জনয ককসবাজার গরেপতার মরগে পাঠিযেছে পলিশ',
  'token': 1448,
  'token_str': 'গ র ে প ত া র'},
 {'score': 0.013577913865447044,
  'sequence': 'লাশ উদধার করে মযনাতদনতের জনয ককসবাজার হতযা মরগে পাঠিযেছে পলিশ',
  'token': 1249,
  'token_str': 'হ ত য া'}]

In [14]:
fill_mask('১৯৭১ সালে বাংলাদেশ ৯ মাস মুক্তিযুদ্ধ করে [MASK] অর্জন করে')

[{'score': 0.016238635405898094,
  'sequence': '১৯৭১ সালে বাংলাদেশ ৯ মাস মকতিযদধ করে বলে অরজন করে',
  'token': 614,
  'token_str': 'ব ল ে'},
 {'score': 0.01370514277368784,
  'sequence': '১৯৭১ সালে বাংলাদেশ ৯ মাস মকতিযদধ করে ও অরজন করে',
  'token': 126,
  'token_str': 'ও'},
 {'score': 0.012568416073918343,
  'sequence': '১৯৭১ সালে বাংলাদেশ ৯ মাস মকতিযদধ করে করে অরজন করে',
  'token': 542,
  'token_str': 'ক র ে'},
 {'score': 0.008793245069682598,
  'sequence': '১৯৭১ সালে বাংলাদেশ ৯ মাস মকতিযদধ করে করা অরজন করে',
  'token': 604,
  'token_str': 'ক র া'},
 {'score': 0.008591611869633198,
  'sequence': '১৯৭১ সালে বাংলাদেশ ৯ মাস মকতিযদধ করে থেকে অরজন করে',
  'token': 586,
  'token_str': 'থ ে ক ে'}]